# PETs/TETs – Hyperledger Aries / PySyft – Manufacturer 3 (Holder) 🛵

In [1]:
%%javascript
document.title = '🛵 Manufacturer3'

<IPython.core.display.Javascript object>

## PART 3: Connect with City to Analyze Data

**What:** Share encrypted data with City agent in a trust- and privacy-preserving manner

**Why:** Share data with City agent (e.g., to obtain funds)

**How:** <br>
1. [Initiate Manufacturer's AgentCommunicationManager (ACM)](#1)
2. [Connect anonymously with the City agent via a multi-use SSI invitation](#2)
3. [Prove Manufacturer3 Agent is a certified manufacturer via VCs](#3)
4. [Establish anonymous Duet Connection to share encrypted data](#4)

**Accompanying Agents and Notebooks:**
* City 🏙️️: `03_connect_with_Manufacturer.ipynb`
* Optional – Manufacturer1 🚗: `03_connect_with_city.ipynb`
* Optional – Manufacturer2 🚛: `03_connect_with_city.ipynb`

---

### 0 - Setup
#### 0.1 - Imports

In [2]:
import os

import numpy as np
import pandas as pd
import syft as sy
import torch
from aries_cloudcontroller import AriesAgentController
import sympc
from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor

from libs.agent_connection_manager import CredentialHolder

#### 0.2 – Variables

In [3]:
# Get relevant details from .env file
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

---

<a id=1></a>

### 1 – Initiate Manufacturer3 Agent
#### 1.1 – Init ACA-PY agent controller

In [4]:
# Setup
agent_controller = AriesAgentController(admin_url,api_key)
print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")

Initialising a controller with admin api at http://manufacturer3-agent:3021 and an api key of adminApiKey


#### 1.2 – Start Webhook Server to enable communication with other agents
@todo: is communication with other agents, or with other docker containers?

In [5]:
# Listen on webhook server
await agent_controller.init_webhook_server(webhook_host, webhook_port)
print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


#### 1.3 – Init ACM Credential Holder

In [6]:
# The CredentialHolder registers relevant webhook servers and event listeners
manufacturer3_agent = CredentialHolder(agent_controller)

# Verify if Manufacturer already has a VC
# (if there are manufacturer credentials, there is no need to execute the notebook)
manufacturer3_agent.get_credentials()

Successfully initiated AgentConnectionManager for a(n) Holder ACA-PY agent


{'results': [{'referent': 'isManufacturer-VC',
   'attrs': {'isManufacturer': 'TRUE',
    'manufacturerName': 'scooterManufacturer',
    'manufacturerCity': 'City3',
    'manufacturerCountry': 'DE'},
   'schema_id': 'h95exdvAfiE4zrFdJtK6X:2:certify-manufacturer:0.0.1',
   'cred_def_id': 'h95exdvAfiE4zrFdJtK6X:3:CL:253012:default',
   'rev_reg_id': None,
   'cred_rev_id': None}]}

---

<a id=2></a>

### 2 – Establish a connection with the City agent 🏙️
A connection with the credential issuer (i.e., the authority agent) must be established before a VC can be received. In this scenario, the Manufacturer3 requests a connection with the Authority to be certified as an official city agency. Thus, the Manufacturer3 agent sends an invitation to the Authority. In real life, the invitation can be shared via video call, phone call, or E-Mail. In this PoC, this is represented by copy and pasting the invitation into the manufacturers' notebooks.

#### 2.1 Join invitation of City agent 🏙️
Copy and paste the multi-use invitation of the city agent, and establish a connection with them.

In [7]:
# Variables
alias = "City Connection"
auto_accept = True

# Receive connection invitation
connection_id = manufacturer3_agent.receive_connection_invitation(alias=alias, auto_accept=auto_accept)

Please enter invitation received by external agent:


Invitation:  {     '@id': '2bfcffe0-d0b2-495d-a929-33537137f191',     '@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation',     'label': 'City-Agency',     'recipientKeys': ['95qrgqn7cjUEPHnUQBPRjAFcEcmvsR2AqxfgcybUAoXj'],     'serviceEndpoint': 'https://ee65-80-134-223-153.ngrok.io', }



---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  9048943b-9300-4704-8ab0-3f167d2384c4
State :  invitation (invitation-received)
Routing State : none
Connection with :  City-Agency
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  9048943b-9300-4704-8ab0-3f167d2384c4
State :  request (request-sent)
Routing State : none
Connection with :  City-Agency
Their Role :  inviter
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Connections Handler
Connection ID :  9048943b-9300-4704-8ab0-3f167d2384c4
State :  response (response-received)
Routing State : none
Connection with :  City-Agency
Their Ro

<div style="font-size: 25px"><center><b>Break Point 2 / 3 / 4</b></center></div>
<div style="font-size: 50px"><center>🛵 ➡️ 🚗 / 🚛 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers. <br> If you have established a connection between the City and all Manufacturers, proceed to the City Notebook's Step 2.2</b></center>

---

<a id=3></a>
### 3 – Create Presentation to Send Proof Presentation

#### 3.1 – Create presentation that satisfies requirements of proof request
Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. To do so, the `prepare_presentation()` function runs through the following steps: 
1. Get all proof requests that were sent through `connection_id`
2. Get the most recent `presentation_exchange_id` and the corresponding `proof_request` from (1)
3. Get the restrictions the City agent defined in `proof_request` from (2)
4. Compare all VCs the Manufacturer3 agent has stored, and find (if available) a VC that satisfies the restrictions from (3)
5. Return a presentation dictionary from a VC from (4) that satisfies all requirements. Generally, a presentation consists of three classes of attributes: <br>
a. `requested_attributes`: Attributes that were signed by an issuer and have been revealed in the presentation process <br>
b. `self_attested_attributes`: Attributes that the prover has self attested to in the presentation object. <br>
c. `requested_predicates` (predicate proofs): Attribute values that have been proven to meet some statement. (TODO: Show how you can parse this information)

In [8]:
presentation, presentation_exchange_id = manufacturer3_agent.prepare_presentation(connection_id)

> Found proof_request with presentation_exchange_id 2fa039e8-fca7-4206-8703-bad518a61114
> Restrictions for a suitable proof: {'isManufacturer': {'requirements': {'schema_id': 'h95exdvAfiE4zrFdJtK6X:2:certify-manufacturer:0.0.1'}, 'request_attr_name': '0_isManufacturer_uuid'}}
> Attribute request for 'isManufacturer' can be satisfied by Credential with VC 'isManufacturer-VC'
> Generate the proof presentation : 
{
    'requested_attributes': {
        '0_isManufacturer_uuid': {
            'cred_id': 'isManufacturer-VC',
            'revealed': True,
        },
    },
    'requested_predicates': {},
    'self_attested_attributes': {},
}


#### 3.2 – Send Presentation

Send the presentation to the recipient of `presentation_exchange_id`.

Overall, the proof-presentation procedure has six steps. **R** represents the Relying Party (here the City agent), and **H** the Holder (i.e., the manufacturers).

| Step | Status | Role | Description | Function |
| --- | --- | --- | --- | --- |
| 1 | `request_sent` |  R | R requests a proof defined in `manufacturer_proof_request` | `send_proof_request()` |
| 2 | `request_received` | H | H receives a proof request from R | - |
| 3 | `presentation-sent` | H | H sends a prepared presentation | `send_proof_presentation()` |
| 4 | `presentation-received` | R | R receives presentation from Step 3 | - |
| 5 | `verified` | R | R verified the proof from Step 4 | `verify_proof_presentation()` |
| 6 | `presentation_acked` | H | Received when Step 5 returns successfully | - |

In [9]:
manufacturer3_agent.send_proof_presentation(presentation_exchange_id, presentation)


---------------------------------------------------------------------
Connection Webhook Event Received: Present-Proof Handler
Connection ID :  9048943b-9300-4704-8ab0-3f167d2384c4
Presentation Exchange ID :  2fa039e8-fca7-4206-8703-bad518a61114
Protocol State :  presentation_sent
Agent Role :  prover
Initiator :  external
---------------------------------------------------------------------

---------------------------------------------------------------------
Connection Webhook Event Received: Present-Proof Handler
Connection ID :  9048943b-9300-4704-8ab0-3f167d2384c4
Presentation Exchange ID :  2fa039e8-fca7-4206-8703-bad518a61114
Protocol State :  presentation_acked
Agent Role :  prover
Initiator :  external
---------------------------------------------------------------------

Presentation Exchange ID: 2fa039e8-fca7-4206-8703-bad518a61114 is acknowledged by Relying Party


---
<a id=4></a>
### 4 – Do Data Science
Assuming that the City agent will acknowledge the proofs and deem them to be correct, proceed by inviting the City agent to a Duet Connection.


#### 4.1 – Establish a Duet Connection with City Agent: Send Duet invitation
Duet is a package that allows you to exchange encrypted data and run privacy-preserving arithmetic operations on them (e.g., through homomorphic encryption or secure multiparty computation).

In [10]:
# Set up connection_id to use for duet connection
manufacturer3_agent._update_connection(connection_id=connection_id, is_duet_connection=True, reset_duet=True)

# Create duet invitation for city agent
duet = sy.launch_duet(credential_exchanger=manufacturer3_agent)

🎤  🎸  ♪♪♪ Starting Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Sending Duet Token bc8ac82f4d7d4ab9723047d7f537746b
♫♫♫ > to Duet Partner City-Agency
♫♫♫ > via Connection ID 9048943b-9300-4704-8ab0-3f167d2384c4
♫♫♫ > Done!

♫♫♫ > STEP 2: Awaiting Duet Token from Duet Partner...

♫♫♫ > DONE! Partner's Duet Token: d001217c2a3dfe8c27ea1960fce1665a
♫♫♫ > Connecting...

♫♫♫ > CONNECTED!

(194, 24)T LIVE STATUS  -  Objects: 1  Requests: 0   Messages: 9  Request Handlers: 0                                


<div style="font-size: 25px"><center><b>Break Point 6 / 7 / 8</b></center></div>
<div style="font-size: 50px"><center>🛵 ➡️ 🚗 / 🚛 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers and run all cells between Steps 3 and 4.1 <br> If you have sent proof presentations from all manufacturers, proceed to the City Notebook's Step 3.3 </b></center>

#### 4.2 - Load encrypted data 🔒🛵 to duet store

In [11]:
# Verify data store of duet
duet.store.pandas  # There should only be an MPC session statement by the City agent

,ID,Tags,Description,object_type
0,<UID: 9a714893d5bd497d9f56c56373ee9df6>,[],,<class 'sympc.session.session.Session'>


Process data before loading it to the duet store. We take a synthetically created dataset of CO2 emission per trip across the City Agent's City (in this case Berlin, Germany).

In [12]:
# Get zipcode data (zipcode data from https://daten.odis-berlin.de/de/dataset/plz/)
df_zipcode = pd.read_csv("data/geo/berlin_zipcodes.csv").rename(columns={"plz":"zipcode"})
df_zipcode.head()

,zipcode
0,10115
1,10117
2,10119
3,10178
4,10179


In [13]:
# Get trip data
df_co2 = pd.read_csv("data/trips/data.csv", index_col=0)
df_co2["hour"] = df_co2.timestamp.apply(lambda x: int(x[11:13]))
df_co2.head()

,i,vehicle_id,manufacturer_id,zipcode,timestamp,latlon,dist,seconds,co2_grams,total_dist,total_seconds,total_co2_grams,timestamp_tripstart,avg_kmperhour,avg_co2perkm,trip_id,hour
0,0,V37335728,V11338,10965,2021-08-19 22:05:16,"(52.4938402, 13.3858854)",0.00,0.0,0.00,0.00,0.0,0.0,2021-08-19 22:05:16,0.0,0.000000,4,22
1,1,V37335728,V11338,10965,2021-08-19 22:05:17,"(52.4938098, 13.3857856)",0.01,1.0,1.35,0.01,1.0,1.0,2021-08-19 22:05:16,36.0,100.000000,4,22
2,2,V37335728,V11338,10965,2021-08-19 22:05:19,"(52.4938978, 13.3855431)",0.02,2.0,2.84,0.03,3.0,4.0,2021-08-19 22:05:16,36.0,133.333333,4,22
3,3,V37335728,V11338,10965,2021-08-19 22:05:20,"(52.4939364, 13.3854384)",0.01,1.0,1.28,0.04,4.0,5.0,2021-08-19 22:05:16,36.0,125.000000,4,22
4,4,V37335728,V11338,10965,2021-08-19 22:05:22,"(52.4940118, 13.3852869)",0.01,2.0,1.52,0.05,6.0,7.0,2021-08-19 22:05:16,30.0,140.000000,4,22


The trip data is then grouped by zipcode to sum the CO2 emission per hour per zipcode.


In [14]:
# Get hourly co2
df_hourly_co2 = df_co2[["zipcode", "hour","co2_grams"]].groupby(["zipcode", "hour"]).sum().reset_index()
df_hourly_co2 = df_hourly_co2.pivot(index=["zipcode"], columns=["hour"])["co2_grams"].replace(np.nan, 0)

# Get matrix that of shape (4085,25)
df_hourly_zipcode = df_zipcode.set_index("zipcode").reindex(columns=list(range(0,24))).replace(np.nan,0)#.reset_index()

# Merge dataframes together
df = df_hourly_zipcode.add(df_hourly_co2, fill_value=0)
print(df.shape)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
zipcode,,,,,,,,,,,,,,,,,,,,,
10115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10117,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10119,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10179,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Then, convert the dataset to a tensor, and upload the tensor with shape (194 x 24) to the duet data store

In [15]:
# Configure tensor
hourly_co2_torch = torch.tensor(df.values)
hourly_co2_torch = hourly_co2_torch.tag("hourly-co2-per-zip_2021-08-19")
hourly_co2_torch = hourly_co2_torch.describe("Total CO2 per Zipcode per Hour on August 19, 2021. Shape: (194, 24)")

# Load tensor to datastore
hourly_co2_torch_pointer = hourly_co2_torch.send(duet, pointable=True)

# Verify datastore
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 9a714893d5bd497d9f56c56373ee9df6>,[],,<class 'sympc.session.session.Session'>
1,<UID: 975984d1f7044e3ab949352856cdb741>,[hourly-co2-per-zip_2021-08-19],"Total CO2 per Zipcode per Hour on August 19, 2...",<class 'torch.Tensor'>


#### 4.3 – Authorize City agent to `.reconstruct()` the data
Authorize the city agent to reconstruct the data once it is shared and joined with other manufacutrers' data

In [16]:
duet.requests.add_handler(action="accept")

---

### 5 – Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.
(Note: the terminating the controller will not terminate the Duet session).

In [17]:
await agent_controller.terminate()


<div style="font-size: 25px"><center><b>Break Point 10 / 11 / 12</b></center></div>
<div style="font-size: 50px"><center>🛵 ➡️ 🚗 / 🚛 / 🏙️ </center></div><br>
<center><b>Please proceed to the remaining Manufacturers and run all cells between Steps 4.2 and 4.3 <br> If you have uploaded all data to the manufacturers' datastored, proceed to the City agent Step 4.2</b></center>

---

## 🔥🔥🔥 You can close this notebook now 🔥🔥🔥